In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

# saving model
model.save('model.savedmodel')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/6
469/469 [==============================] - 10s 7ms/step - loss: 0.6511 - sparse_categorical_accuracy: 0.8163 - val_loss: 0.1920 - val_sparse_categorical_accuracy: 0.9439
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1823 - sparse_categorical_accuracy: 0.9484 - val_loss: 0.1383 - val_sparse_categorical_accuracy: 0.9592
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1248 - sparse_categorical_accuracy: 0.9647 - val_loss: 0.1109 - val_sparse_categorical_accuracy: 0.9671
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0920 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.0931 - val_sparse_categorical_accuracy: 0.9735
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0741 - sparse_categorical_accuracy: 0.9792 - val_loss: 0.0857 - val_sparse_categorical_

INFO:tensorflow:Assets written to: model.savedmodel/assets
